# Import Software Packages

In [23]:
import warnings  # Suppress warnings
from time import time  # Calculate training time.

import matplotlib
import numpy as np
import pandas as pd
import sklearn
from matplotlib import pyplot as plt
from sklearn import datasets

warnings.filterwarnings("ignore")

# Summarize software libraries used.
print("Libraries used in this project:")
print("- NumPy {}".format(np.__version__))
print("- Matplotlib {}".format(matplotlib.__version__))
print("- scikit-learn {}\n".format(sklearn.__version__))

Libraries used in this project:
- NumPy 1.24.3
- Matplotlib 3.7.2
- scikit-learn 1.3.2

